This notebook takes the updated TNS csv file and "cleans" it by removing SNe without discovery dates, and averaging the reported RAs and Decs.

Then converts the desired entries into a region file.

In [1]:
#Import Packages

import numpy as np
import os
from astropy.table import Table


In [2]:
##### Open downloaded TNS csv file and "clean" it by removing SNe without discovery dates, and averaging the 
#  reported RAs and Decs.
#TNS File Rows: "ID","Name","RA","DEC","Obj. Type","Redshift","Host Name","Host Redshift","Reporting Group/s",
#  "Discovery Data Source/s","Classifying Group/s","Associated Group/s","Disc. Internal Name","Disc. Instrument/s",
#  "Class. Instrument/s","TNS AT","Public","End Prop. Period","Discovery Mag/Flux","Discovery Filter","Discovery
#  Date (UT)","Sender","Remarks","Discovery Bibcode","Classification Bibcodes","Ext. catalog/s"

date = "Dec_30_2023"

tnsOriginal  = '../Data/TNS/tns_search_ALL_TYPED_' + date + '.csv'
tnsInProcess = '../Data/1.TNS_' + date + '_In_Process.csv'
tnsClean     = '../Data/1.TNSCleaned.csv'
tnsRegion    = '../Data/1.TNS.reg'

disc, ra, dec = np.genfromtxt(tnsOriginal, dtype=str,unpack=True, delimiter = '","',usecols=(20,2,3))
i = 0

# Take entries from original TNS file with discovery dates and copy over to new file.

with open(tnsOriginal, 'r') as f1:
    lines = f1.readlines()
with open(tnsInProcess, 'w') as f2:
    for line in lines:
        if disc[i] != '' and ra[i] != '' and dec[i] != '':
            f2.write(line)
        i += 1
f1.close()
f2.close()

# Take RA & Dec values and convert them into decimal degree format, then averages reported coordinates for each entry.

name = np.genfromtxt(tnsInProcess, dtype=str,unpack=True, delimiter = ",",usecols=(1))
name = [x.replace(" ","") for x in name]
name = [x.replace('"',"") for x in name]
galaxy = np.genfromtxt(tnsInProcess, dtype=str,unpack=True, delimiter = ",",usecols=(6))
SNtype = np.genfromtxt(tnsInProcess, dtype=str,unpack=True, delimiter = '","',usecols=(4))
SNtype = [x.replace('SN ',"") for x in SNtype]
# for i in range(len(SNtype)):
#     if SNtype[i] == " ":
#         print("empty")
disc, ra, dec = np.genfromtxt(tnsInProcess, dtype=str,unpack=True, delimiter = '","',usecols=(20,2,3))
print(len(disc))

18262


In [3]:
raVals, decVals, raDeg, decDeg, sign = [],[],[],[],[]
raDegrees, decDegrees = [],[]
for i in range(len(ra)):
    vals = [x.strip() for x in ra[i].strip("''").split(',')]
    for j in range(len(vals)):
        if vals[j] != '00:00:00.000':
            valString = vals[j] 
            parts = [x.strip() for x in valString.split(':')]
            if len(parts) == 3:
                deg = (float(parts[0]) + (float(parts[1])/60.0) + (float(parts[2])/3600.0)) * 15
                raDeg.append(deg)
            elif len(parts) == 1:
                deg = float(parts[0])*15
                raDeg.append(deg)
            else: 
                deg = (float(parts[0]) + (float(parts[1])/60.0))*15
                raDeg.append(deg) 
        else:
            pass
    degVal = np.sum(raDeg)/len(raDeg)
    degree = degVal
    raDegrees.append(degree)
    hmsVal = degVal / 15.0
    minVal = hmsVal % 1.0
    hourVal = hmsVal-minVal
    hourVal = str(hourVal).split('.')[0]
    minVal  = minVal * 60
    secVal = minVal % 1.0
    minVal = str(minVal).split('.')[0]
    if(len(minVal)) < 2:
        minVal = '0' + minVal
    secVal *= 60
    secVal = str(round(secVal, 2))
    frontSec = secVal.split('.')[0]
    if len(frontSec) < 2:
        secVal = "0" + secVal
    hmsString = (hourVal) + ':' + (minVal) + ':' + secVal
    raVals.append(hmsString)
    raDeg = []
with open('../Data/1.TNSRaDeg.txt', 'w') as text_file:
    for i in range(len(ra)):
        print(name[i],raDegrees[i],file=text_file)
text_file.close()

for i in range(len(dec)):
    vals = [x.strip() for x in dec[i].strip("''").split(',')]
    for j in range(len(vals)):
        valString = vals[j] 
        parts = [x.strip() for x in valString.split(':')]
        hourString = parts[0]
        sign = hourString[:1]
        hour = hourString[1:]
        if len(parts) == 3:
            deg = (float(hour) + (float(parts[1])/60.0) + (float(parts[2])/3600.0)) 
            decDeg.append(deg)
        elif len(parts) == 1:
            deg = float(hour)
            decDeg.append(deg)
        else: 
            deg = (float(hour) + (float(parts[1])/60.0))
            decDeg.append(deg)
    degVal = np.sum(decDeg)/len(decDeg) 
    degree = degVal
    if sign == '-':
        degree *= -1
    decDegrees.append(degree)
    dmsVal = degVal 
    minVal = dmsVal % 1.0
    degreeVal = dmsVal-minVal
    degreeVal = str(degreeVal).split('.')[0]
    minVal  = minVal * 60
    secVal = minVal % 1.0
    minVal = str(minVal).split('.')[0]
    if(len(minVal)) < 2:
        minVal = '0' + minVal
    secVal *= 60
    secVal = str(round(secVal, 2))
    frontSec = secVal.split('.')[0]
    if len(frontSec) < 2:
        secVal = "0" + secVal
    if sign == '-':
        dmsString = '-' + (degreeVal) + ':' + (minVal) + ':' + secVal
    else:
        dmsString = '+' +(degreeVal) + ':' + (minVal) + ':' + secVal
    decVals.append(dmsString)
    decDeg = []
with open('../Data/1.TNSDecDeg.txt', 'w') as text_file:
    for i in range(len(ra)):
        print(name[i],decDegrees[i],file=text_file)
text_file.close()
print(SNtype)

os.remove(tnsInProcess)

['Ia', 'Ia', 'II', 'II', 'Ia', 'Ia', 'Ia', 'Ia', 'Ia', 'Ia-91T-like', 'Ia', 'Ia', 'Ia', 'Ia', 'Ia', 'Ic', 'Ia', 'Ia', 'SLSN-I', 'Ia', 'Ia', 'Ia-91T-like', 'Ia', 'Ia', 'Ia', 'Ia', 'Ia', 'Ia', 'Ia', 'II', 'Ia', 'II', 'Ia', 'Ia', 'IIb', 'Ia', 'II', 'Ia', 'Ia', 'Ia', 'Ia', 'Ia', 'Ib', 'Ia', 'Ia-91T-like', 'Ia', 'Ia-91T-like', 'Ia', 'Ia', 'Ia', 'Iax[02cx-like]', 'Ia', 'II', 'Ia', 'Ic-BL', 'Ia', 'Ic-BL', 'SLSN-II', 'II', 'Ia', 'II', 'Ia', 'Ia-pec', 'IIP', 'II', 'Ia', 'Ia', 'II', 'Ia', 'Ia', 'IIb', 'Ia', 'IIP', 'IIn', 'Ia', 'Ib', 'Ia', 'II', 'Ia', 'II', 'Ia', 'Ia', 'Ia', 'IIn', 'II', 'Ia', 'Ia', 'Ia', 'Ia', 'II', 'Ia', 'Ia', 'Ia', 'Ia', 'Ia', 'II', 'Ia', 'Ia', 'Ia', 'Ia', 'II', 'II', 'Ib', 'Ia', 'II', 'Ia', 'Ia', 'Ia-91bg-like', 'IIn', 'Ia', 'Ia', 'Ic', 'Ia', 'Ia', 'II', 'Ia', 'Ia', 'Ia', 'II', 'Ia', 'Ia', 'Ia', 'II', 'IIn', 'Ia', 'Ia', 'II', 'Ia', 'Ia', 'Ia-pec', 'Ia', 'Ia', 'Ia-91T-like', 'Ia', 'Ia', 'IIn', 'Ia', 'II', 'IIn', 'Ia', 'Ia-91T-like', 'II', 'Ia', 'II', 'Ia', 'Ia', 'Ia', 'Ia', 'I

In [4]:
#Copy over cleaned TNS file to final file with unclassified SNe specified as such
        
# SNtype = [SN.replace('', 'unclassified') for SN in SNtype]
cleanTable = Table([name,galaxy,raVals,decVals,raDegrees, decDegrees, SNtype, disc], names=["Name","Host","RA hms","Dec dms","RA dds","Dec dds","Type","DiscDate"])
cleanTable.write(tnsClean, overwrite = True)


In [5]:
# Create region file for DS9 visualization of SNe using the following DS9 format
#text x y {Your Text Here}

radius = 3
with open(tnsRegion, 'w') as text_file:
    print('global color = magenta', file=text_file)
    for i in range(len(raVals)):
        print('circle',raVals[i],decVals[i],radius,'# text=',name[i], file=text_file)
text_file.close()

In [6]:
# i = 0
# with open('regionsThatWorked.txt', 'r') as f1:
#     lines = f1.readlines()
#     for line in lines:
#         i += 1
# f1.close()
# print(i)
# print(len(ra))

In [7]:
print("finished")

finished
